In [ ]:
import torch

# Enable expandable segments (run before initializing tensors)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Monitor memory usage
print(torch.cuda.memory_summary())

In [ ]:
from pathlib import Path

model_name = 'yolo11l.pt' # The starting model weights
num_epochs = 100
image_size = 960
dataset = Path('../../datasets/player_detection/')
data_yaml_path = dataset / 'data.yaml'
print(data_yaml_path.exists())
run_name = 'v_1_1large_long'
project_name = 'Pitch_Analyzer'

In [ ]:
#Optional enable mlflow
from ultralytics import settings
import mlflow

settings.update({"mlflow": True})
mlflow.set_experiment(project_name)
mlflow.autolog()

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)

print(f"Starting training with model: {model_name}")
print(f"Dataset config: {data_yaml_path}")
print(f"Epochs: {num_epochs}, Image Size: {image_size}")
print(f"Run Name: {run_name}")

torch.cuda.empty_cache()

model = YOLO(model_name)

results = model.train(
    data=data_yaml_path,
    epochs=num_epochs,
    imgsz=image_size,
    batch=0.8, # Automatically use 80% of GPU
    project=project_name,
    name=run_name
)

torch.cuda.empty_cache()

In [ ]:
# Move the trained model
import shutil
# Path where YOLO saves the weights
trained_model_path = Path(f"../runs/detect/{run_name}/weights/best.pt")
output_model_dir = Path(f"../models/player_detection/{run_name}")
# Create your custom models directory if it doesn't exist
os.makedirs(output_model_dir, exist_ok=True)

# Move the trained model
shutil.copy(trained_model_path, f"{output_model_dir}/best.pt")
print(f"Model saved to {output_model_dir}/best.pt")